In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML


# Helper Functions

import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def sort_line(line1,line2):
    
    for x1,y1,x2,y2 in line1:
        mn1 = min(x1,x2)
    for x1,y1,x2,y2 in line2:
        mn2 = min(x1,x2)
    return mn1 < mn2 
    
    
def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """    

    bottom_left = []
    bottom_right = []
    top_left = []
    top_right = []
    c1 = 0
    c2 = 0
    m1 = 0
    m2 = 0
    count1 = 0
    count2 = 0
    for line in lines:        
        for x1,y1,x2,y2 in line:            
            #cv2.line(img, (x1, y1), (x2, y2), color, thickness)
            #print(line)
            m = (y1-y2)/(x1-x2)
            if y1 > y2:                
                if x1!=x2 and abs(m)>0.5 and abs(m) < 0.9:
                    m1 += m
                    c1 += y1 - m*x1
                    count1 = count1 + 1                    
            else:
                if y1 < y2:                
                    if x1 != x2 and abs(m) > 0.5 and abs(m) < 0.9 :                        
                        m2 += m
                        c2 += y1 - m*x1
                        count2 = count2 + 1
                        
    if(count1 == 0 or count2 == 0):
        return
    
    m1 = m1 / count1
    c1 = c1 / count1
    m2 = m2 / count2
    c2 = c2 / count2
    
    bottom_left = (int((img.shape[0]-c1)/m1),img.shape[0]);
    bottom_right = (int((img.shape[0]-c2)/m2),img.shape[0]);
    top_left = (int((320-c1)/m1),320);
    top_right = (int((320-c2)/m2),320);
    
    cv2.line(img, (bottom_left[0],bottom_left[1]), (top_left[0],top_left[1]), color, 10)
    cv2.line(img, (bottom_right[0],bottom_right[1]), (top_right[0],top_right[1]), color, 10)
            
    
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """    
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

def show_image(img,gray=0):
    return
    if gray == 1:
        plt.imshow(img, cmap='gray')        
    else:
        plt.imshow(img)    
    plt.show()
    
    
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)    

    gray = grayscale(image)    
    show_image(gray,1)
    #plt.imshow(gray, cmap='gray')        
    # Define a kernel size and apply Gaussian smoothing
    kernel_size = 3
    blur_gray = gaussian_blur(gray,kernel_size)
    
    #show_image(blur_gray,1)
    #plt.imshow(blur_gray, cmap='gray')        
    # Define our parameters for Canny and apply
    low_threshold = 50
    high_threshold = 150
    edges = canny(blur_gray, low_threshold, high_threshold)   
    
    
    # Next we'll create a masked edges image using cv2.fillPoly()
    mask = np.zeros_like(edges)   
    ignore_mask_color = 255   

    # This time we are defining a four sided polygon to mask
    imshape = image.shape
    vertices = np.array([[(0,imshape[0]),(460, 320), (530, 320), (imshape[1],imshape[0])]], dtype=np.int32)
    
    masked_edges = region_of_interest(edges,vertices);
    
    # Define the Hough transform parameters
    # Make a blank the same size as our image to draw on
    rho = 2 # distance resolution in pixels of the Hough grid
    theta = (np.pi/180) # angular resolution in radians of the Hough grid
    threshold = 30     # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 100 #minimum number of pixels making up a line
    max_line_gap = 160    # maximum gap in pixels between connectable line segments

    # Run Hough on edge detected image
    line_image = hough_lines(masked_edges, rho, theta, threshold, min_line_length, max_line_gap)
    

    # Draw the lines on the edge image
    lines_edges = cv2.addWeighted(image, 0.8, line_image, 1, 0) 
    #show_image(lines_edges,0)
    #plt.imshow(line_image)  
    return lines_edges

image = mpimg.imread('solidWhiteCurve.jpg')
plt.imshow(image)
process_image(image)
'''
white_output = 'white.mp4'
#clip1 = VideoFileClip("challenge.mp4")
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))
'''

ImportError: No module named 'cv2'

In [11]:
# My short comings are not having a good understanding of various values we set. Like the threshold, min_line_gap etc. 
# But I understand a bit which I feel like not enough. I have managed to detect line but my lines are a bit shaky.
# The reason for that is I think the process I took to draw line. I have first summed all the slopes and intersect for different
# points for both lines. Then took average of them. After that I used these values to find bottom x,y and top x,y for both 
# lane. To find the values my logic is following,
# I know that at the bottom Y is maximum height of the image. So with the intersect, slope and Y I can find the X. 
# At the top Y is 320 which I defined as the height for my region of interest. And then with same approach find X of it.
# So now I have two pair of (X,Y). With them I drew the line. But I think this process is a bit faulty. Or my edge
# detection paramters are not working properly :( I tried increasing the threshold and line gap and line lenth.
# Also set kernel size to 3. but it didnt improve much. Would really Like to see the actual solution for this project.

## 